In [10]:
import psycopg2
import pandas as pd
import numpy as np
import psycopg2.extras as extras
from dotenv import load_dotenv
import os

In [7]:
# End goal: Make Flask API and Library 
# End goal of this step: Load data into Database

## Connect to database

In [8]:
# Connect to DB
load_dotenv()
connection = psycopg2.connect(
    database = "postgres",
    user = "census",
    password = os.environ.get('PASSWORD'),
    host = os.environ.get('HOST'),
    port = 5432
    )
cursor=connection.cursor()

In [9]:
# Test Connection
cursor=connection.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

('demographics',)


## Insert Demographics Table into DB

In [43]:
# Create Table
command = """CREATE TABLE demographics (
        Zip_Code VARCHAR(255) PRIMARY KEY,
        Occupied_housing_units INTEGER NOT NULL,
        White INTEGER NOT NULL,
        Black_or_African_American INTEGER NOT NULL,
        American_Indian_and_Alaska_Native INTEGER NOT NULL,
        Asian INTEGER NOT NULL,
        Native_Hawaiian_and_Other_Pacific_Islander INTEGER NOT NULL,
        Some_other_race INTEGER NOT NULL,
        Two_or_more_races INTEGER NOT NULL,
        Hispanic_or_Latino_origin INTEGER NOT NULL,
        White_alone_not_Hispanic_or_Latino INTEGER NOT NULL,
        Under_35_years INTEGER NOT NULL,
        age_35_to_44_years INTEGER NOT NULL,
        age_45_to_54_years INTEGER NOT NULL,
        age_55_to_64_years INTEGER NOT NULL,
        age_65_to_74_years INTEGER NOT NULL,
        age_75_to_84_years INTEGER NOT NULL,
        age_85_years_and_over INTEGER NOT NULL,
        Less_than_high_school_graduate INTEGER NOT NULL,
        High_school_graduate INTEGER NOT NULL,
        Some_college INTEGER NOT NULL,
        Bachelors_degree_or_higher INTEGER NOT NULL)"""
try:
    cursor.execute(command)
    cursor.close()
    connection.commit()
    print('Table Successfully Created')
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Table Successfully Created


In [46]:
# Check data in csv
demographicsdf = pd.read_csv('CleanedDemographics2021.csv', dtype={'Zip Code': 'str'})
demographicsdf.head()

,Zip Code,Occupied housing units,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some other race,Two or more races,Hispanic or Latino origin,...,35 to 44 years,45 to 54 years,55 to 64 years,65 to 74 years,75 to 84 years,85 years and over,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher
0,00601,5397,4640,28,4,0,0,500,225,5364,...,768,998,1242,1065,600,189,1685,1645,1050,1017
1,00602,12858,7735,222,12,0,0,501,4388,12385,...,1585,2233,3065,2926,1669,558,4748,3324,2381,2405
2,00603,19295,14013,883,26,19,0,2836,1518,18982,...,2709,3105,4029,4011,2525,608,5427,5478,3648,4742
3,00606,1968,1053,0,11,0,0,793,111,1962,...,236,427,363,410,219,152,851,598,363,156
4,00610,8934,4907,198,0,0,0,571,3258,8658,...,1123,1659,2016,1793,1106,277,2750,2607,1899,1678


In [47]:
# Clean up columns
newdemcols = []
demcols = demographicsdf.columns.values
for col in demcols:
    newcol = col.replace(",", "" )
    newcol = newcol.replace("'", "" )
    newcol = newcol.replace(" ", "_" )
    if newcol[0].isnumeric():
        newcol = 'age_' + newcol
    newdemcols.append(newcol)
newdemcols[-3] = 'High_school_graduate'
newdemcols[-2] = 'Some_college'
demographicsdf.columns = newdemcols
demographicsdf.head()

,Zip_Code,Occupied_housing_units,White,Black_or_African_American,American_Indian_and_Alaska_Native,Asian,Native_Hawaiian_and_Other_Pacific_Islander,Some_other_race,Two_or_more_races,Hispanic_or_Latino_origin,...,age_35_to_44_years,age_45_to_54_years,age_55_to_64_years,age_65_to_74_years,age_75_to_84_years,age_85_years_and_over,Less_than_high_school_graduate,High_school_graduate,Some_college,Bachelors_degree_or_higher
0,00601,5397,4640,28,4,0,0,500,225,5364,...,768,998,1242,1065,600,189,1685,1645,1050,1017
1,00602,12858,7735,222,12,0,0,501,4388,12385,...,1585,2233,3065,2926,1669,558,4748,3324,2381,2405
2,00603,19295,14013,883,26,19,0,2836,1518,18982,...,2709,3105,4029,4011,2525,608,5427,5478,3648,4742
3,00606,1968,1053,0,11,0,0,793,111,1962,...,236,427,363,410,219,152,851,598,363,156
4,00610,8934,4907,198,0,0,0,571,3258,8658,...,1123,1659,2016,1793,1106,277,2750,2607,1899,1678


In [48]:
# Method for filling table
def execute_table(connection, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = connection.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [49]:
# Execute table creation
execute_table(connection, demographicsdf, 'demographics')

the dataframe is inserted


In [51]:
# Test Basic Query
cursor = connection.cursor()
cursor.execute("""
    SELECT * 
    FROM demographics d
    WHERE d.Zip_Code = %s;
    """,
    ['91007',])
zipdata = cursor.fetchall()
for row in zipdata:
    print(row)
cursor.close()

('91007', 11708, 3867, 218, 30, 6458, 40, 623, 472, 1692, 3201, 1474, 2105, 2677, 2273, 1619, 775, 785, 745, 1809, 2269, 6885)


In [10]:
#Close connections when done

cursor.close()
connection.close()